# Fine tuning 3 resnet 34 fusion model 

In [1]:
import os
import sys
import glob
import shutil
import random
import pickle
import numpy as np
from PIL import Image
import time
import copy
from tqdm import tqdm
from pathlib import Path

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.utils.data
from torchvision.models import resnet34
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler

%reload_ext autoreload
%autoreload 2
%matplotlib inline

DATA_PATH = Path('/home/rene/data/idc')

# Add the src directory for functions
src_dir = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'src')
print(src_dir)
sys.path.append(src_dir)

# import my functions:
from functions import*
from models import*

# torch.cuda.set_device(1)
print(torch.cuda.is_available())
print(torch.cuda.current_device())

/home/rene/code/idc/src
True
0


## Fusion 1 Result

In [2]:
model_list = [resnet34, resnet34, resnet34]
model_name_list = ['resnet34_0', 'resnet34_5', 'resnet34_2']

batch_size = 16
num_workers = 6
save_path = '/media/rene/Data/data/idc/models'
PATH = '/media/rene/Data/data/idc'
input_size = 6

pretrained_model_list = []
for idx, model_arch in enumerate(model_list):
    model_ft = model_arch(pretrained=False)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()
    model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name_list[idx])))
    pretrained_model_list.append(model_ft)

In [3]:
dataloaders, dataset_sizes= make_batch_gen(PATH, batch_size, num_workers, 
                                                        valid_name='valid', test_name='test', size=197)

fusion_model = Fusion1Hardcode(pretrained_model_list)
fusion_model = fusion_model.cuda()

fusion_model.load_state_dict(torch.load(os.path.join(save_path, 'Fusion1_1')))
batch_size = 8
num_workers = 6
criterion = nn.CrossEntropyLoss()

valid_loss, valid_acc = eval_model(model_ft, dataloaders['valid'], dataset_sizes['valid'], criterion)
print('Validation Fusion1_1: ', valid_loss, valid_acc)

test_loss, test_acc = eval_model(model_ft, dataloaders['test'], dataset_sizes['test'], criterion)
print('Test Fusion1_1: ' , test_loss, test_acc)

Loss: 0.4197 Acc: 0.8521
Validation Fusion1_1:  0.4196543438838542 0.8521463402916236
Loss: 0.4202 Acc: 0.8528
Test Fusion1_1:  0.4201910931596171 0.8527949266136587


# Fusion 2 stage

In [3]:
model_list = [resnet34, resnet34, resnet34]
model_name_list = ['resnet34_0', 'resnet34_5', 'resnet34_2']

batch_size = 32
num_workers = 6
input_size = 6

# save_path = '/media/rene/Data/data/idc/models'
# PATH = '/media/rene/Data/data/idc'
save_path =  Path(DATA_PATH, 'models')
PATH = DATA_PATH

pretrained_model_list = []
for idx, model_arch in enumerate(model_list):
    model_ft = model_arch(pretrained=False)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()
    model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name_list[idx])))
    pretrained_model_list.append(model_ft)

model_ft = Fusion2Hardcode(pretrained_model_list)

dataloaders, dataset_sizes= make_batch_gen(PATH, batch_size, num_workers, 
                                                        valid_name='valid', test_name='test', size=197)

fusion_model = Fusion2Hardcode(pretrained_model_list)
fusion_model = fusion_model.cuda()

fusion_model.load_state_dict(torch.load(os.path.join(save_path, 'Fusion2_2s2-test-full-data')))
batch_size = 8
num_workers = 6
criterion = nn.CrossEntropyLoss()

valid_loss, valid_acc = eval_model(fusion_model, dataloaders['valid'], dataset_sizes['valid'], criterion)
print('Validation Fusion2_2s2-test-full-data: ', valid_loss, valid_acc)

valid_loss, valid_acc = eval_model(fusion_model, dataloaders['valid'], dataset_sizes['valid'], criterion)
print('Validation Fusion2_2s2-test-full-data: ', valid_loss, valid_acc)

Loss: 0.2222 Acc: 0.9092
Validation Fusion2_2s2-test-full-data:  0.22221683784354648 0.9092219366307142
Loss: 0.2222 Acc: 0.9092
Validation Fusion2_2s2-test-full-data:  0.22221683610533421 0.9092219366307142


## Sensitivity/Specificity

In [6]:
all_labels, all_preds = get_preds(fusion_model, dataloaders['valid'], dataset_sizes['valid'])
metrics = get_metrics_bin(all_labels, all_preds)
print(f'Validation sensitivity: {metrics["TPR"]}, specificity: {metrics["TNR"]}, Acc: {metrics["ACC"]}')

all_labels, all_preds = get_preds(fusion_model, dataloaders['test'], dataset_sizes['test'])
metrics = get_metrics_bin(all_labels, all_preds)
print(f'Test sensitivity: {metrics["TPR"]}, specificity: {metrics["TNR"]}, Acc: {metrics["ACC"]}')

Validation sensitivity: 0.9293772265470379, specificity: 0.8552108194112967, Acc: 0.9092219366307142
Test sensitivity: 0.9309866137828459, specificity: 0.851063829787234, Acc: 0.9091498714838214


# Fusion 2 -test-

In [9]:
model_list = [resnet34, resnet34, resnet34]
model_name_list = ['resnet34_0', 'resnet34_5', 'resnet34_2']

batch_size = 16
num_workers = 6
# save_path = '/media/rene/Data/data/idc/models'
# PATH = '/media/rene/Data/data/idc'

save_path =  Path(DATA, 'models')
PATH = DATA_PATH

input_size = 6

pretrained_model_list = []
for idx, model_arch in enumerate(model_list):
    model_ft = model_arch(pretrained=False)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()
    model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name_list[idx])))
    pretrained_model_list.append(model_ft)


dataloaders, dataset_sizes= make_batch_gen(PATH, batch_size, num_workers, 
                                                        valid_name='valid', test_name='test', size=197)

fusion_model = Fusion2Hardcode(pretrained_model_list)
fusion_model = fusion_model.cuda()

fusion_model.load_state_dict(torch.load(os.path.join(save_path, 'Fusion2_1-test-full-data')))
batch_size = 8
num_workers = 6
criterion = nn.CrossEntropyLoss()

In [3]:
valid_loss, valid_acc = eval_model(model_ft, dataloaders['valid'], dataset_sizes['valid'], criterion)
print('Validation Fusion2_1: ', valid_loss, valid_acc)

test_loss, test_acc = eval_model(model_ft, dataloaders['test'], dataset_sizes['test'], criterion)
print('Test Fusion2_1: ' , test_loss, test_acc)

Loss: 0.4888 Acc: 0.8933
Validation Fusion2_1:  0.48884277451232083 0.8932955391674073
Loss: 0.4832 Acc: 0.8954
Test Fusion2_1:  0.4832239666094465 0.8954094501429292


# FUCK

In [3]:
all_labels, all_preds = get_preds_fusion(fusion_model, pretrained_model_list, dataloaders['valid'], dataset_sizes['valid'])
metrics = get_metrics_bin(all_labels, all_preds)
print(f'Validation sensitivity: {metrics["TPR"]}, specificity: {metrics["TNR"]}, Acc: {metrics["ACC"]}')

all_labels, all_preds = get_preds_fusion(fusion_model, pretrained_model_list, dataloaders['test'], dataset_sizes['test'])
metrics = get_metrics_bin(all_labels, all_preds)
print(f'Test sensitivity: {metrics["TPR"]}, specificity: {metrics["TNR"]}, Acc: {metrics["ACC"]}')

ValueError: Expected 4D tensor as input, got 2D tensor instead.

# Useless:::

## Fusion 2 Result

In [7]:
model_list = [resnet34, resnet34, resnet34]
model_name_list = ['resnet34_0', 'resnet34_5', 'resnet34_2']

batch_size = 16
num_workers = 6
save_path = '/media/rene/Data/data/idc/models'
PATH = '/media/rene/Data/data/idc/sample'


pretrained_model_list = []
for idx, model_arch in enumerate(model_list):
    model_ft = model_arch(pretrained=False)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()
    model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name_list[idx])))
    pretrained_model_list.append(model_ft)


dataloaders, dataset_sizes= make_batch_gen(PATH, batch_size, num_workers, 
                                                        valid_name='valid', test_name='test', size=197)

fusion_model = Fusion2Hardcode(pretrained_model_list)
fusion_model = fusion_model.cuda()

fusion_model.load_state_dict(torch.load(os.path.join(save_path, 'Fusion2_1')))
batch_size = 32
num_workers = 6
criterion = nn.CrossEntropyLoss()

valid_loss, valid_acc = eval_model(model_ft, dataloaders['valid'], dataset_sizes['valid'], criterion)
print('Validation Fusion2_1: ', valid_loss, valid_acc)

test_loss, test_acc = eval_model(model_ft, dataloaders['test'], dataset_sizes['test'], criterion)
print('Test Fusion2_1: ' , test_loss, test_acc)

Loss: 0.9721 Acc: 0.5387
Validation Fusion2_1:  0.9721003385875616 0.5386833253243632
Loss: 0.9684 Acc: 0.5454
Test Fusion2_1:  0.9683997267315675 0.545410860163383


In [7]:
model_list = [resnet34, resnet34, resnet34]
model_name_list = ['resnet34_0', 'resnet34_5', 'resnet34_2']

batch_size = 16
num_workers = 6
save_path = '/media/rene/Data/data/idc/models'
PATH = '/media/rene/Data/data/idc'
input_size = 6

dataloaders, dataset_sizes = make_batch_gen(PATH, batch_size, num_workers, 
                                            valid_name='valid', test_name='test', size=197)

pretrained_model_list = []
for idx, model_arch in enumerate(model_list):
    model_ft = model_arch(pretrained=False)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()
    model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name_list[idx])))
    pretrained_model_list.append(model_ft)

In [ ]:
dataloaders, dataset_sizes= make_batch_gen(PATH, batch_size, num_workers, 
                                                        valid_name='valid', test_name='test', size=197)

fusion_model = Fusion2Hardcode(pretrained_model_list)
fusion_model = fusion_model.cuda()

fusion_model.load_state_dict(torch.load(os.path.join(save_path, 'Fusion2_1')))
batch_size = 8
num_workers = 6
criterion = nn.CrossEntropyLoss()

valid_loss, valid_acc = eval_model(model_ft, dataloaders['valid'], dataset_sizes['valid'], criterion)
print('Validation Fusion2_1: ', valid_loss, valid_acc)

test_loss, test_acc = eval_model(model_ft, dataloaders['test'], dataset_sizes['test'], criterion)
print('Test Fusion2_1: ' , test_loss, test_acc)

## Debug test/val perf

In [5]:
dataloaders, dataset_sizes= make_batch_gen(PATH, batch_size, num_workers, 
                                                        valid_name='valid', test_name='test', size=197)

fusion_model = Fusion2Hardcode(pretrained_model_list)
fusion_model = fusion_model.cuda()

fusion_model.load_state_dict(torch.load(os.path.join(save_path, 'Fusion2_1')))
batch_size = 8
num_workers = 6
criterion = nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ft.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

best_acc, epoch_acc, epoch_loss = eval_model_testing(model_ft, criterion, optimizer, scheduler, 
                                                     dataloaders, dataset_sizes)
print('Validation Fusion2_1: ', best_acc, epoch_acc, epoch_loss)

best_acc, epoch_acc, epoch_loss = eval_model_testing(model_ft, criterion, optimizer, scheduler, 
                                                     dataloaders, dataset_sizes)
print('Test Fusion2_1: ' , best_acc, epoch_acc, epoch_loss)

valid Loss: 0.9601 Acc: 0.5570
Validation Fusion2_1:  0.5570155420500132 0.5570155420500132 0.9601346823262482
valid Loss: 0.9601 Acc: 0.5570
Test Fusion2_1:  0.5570155420500132 0.5570155420500132 0.9601346820728186


In [6]:
all_labels, all_preds = get_preds_fusion(fusion_model, model_list_ft, dataloaders['valid'], dataset_sizes['valid'])
metrics = get_metrics_bin(all_labels, all_preds)
print(f'Validation sensitivity: {metrics["TPR"]}, specificity: {metrics["TNR"]}, Acc: {metrics["ACC"]}')

all_labels, all_preds = get_preds_fusion(fusion_model, model_list_ft, dataloaders['test'], dataset_sizes['test'])
metrics = get_metrics_bin(all_labels, all_preds)
print(f'Test sensitivity: {metrics["TPR"]}, specificity: {metrics["TNR"]}, Acc: {metrics["ACC"]}')